In [1]:
import networkx as nx
import pickle
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import combinations, groupby
import datetime
import sys

In [2]:
entries = os.listdir('grafi_da_eseguire/')
entries_prof = os.listdir('grafi_prof/')

In [3]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [4]:
def load_graph(name):
    G = pickle.load(open("grafi_da_eseguire/"+name, 'rb'))
    num_of_nodes = G.number_of_nodes()
    num_of_edges = G.number_of_edges()

    s = (num_of_nodes, num_of_edges)
    graph = np.zeros(s, dtype='uint8') # matrix of zeros

    for source, target in G.edges():
        graph[source][target] = G[source][target]['weight']

    return graph

In [5]:
def load_graph_prof(name):
    with open('grafi_prof/' + name, 'r') as f:
        first_line = f.readline().strip('\n')
        second_line = f.readline().strip('\n')
        third_line = f.readline().strip('\n')

        values = first_line.split()
        n_nodes = int(values[2])

        s = (n_nodes, n_nodes)
        graph = np.zeros(s) # matrix of zeros

        values = second_line.split()
        source = int(values[1])

        values = third_line.split()
        sink = int(values[1])

    file1 = open('grafi_prof/' + name, 'r')
    Lines = file1.readlines()

    count = 0

    for line in Lines:
        count += 1
        current_line = line.strip()

        if len(current_line) > 0 and current_line[0] != 'c' and current_line[0] != 'p':
            if current_line[0] == 'a':
                values = current_line.split()
                node1 = int(values[1])
                node2 = int(values[2])
                weight = int(values[3])
                graph[node1-1][node2-1] = weight

    return graph, source, sink

# Edmonds-Karp algorithm

In [6]:
#Edmonds-Karp Algorithm
def edmonds_karp(C, s, t):
        n = len(C) # C is the capacity matrix
        F = [[0] * n for i in range(n)]
        path = bfs(C, F, s, t)
        while path != None:
            flow = min(C[u][v] - F[u][v] for u,v in path)
            for u,v in path:
                F[u][v] += flow
                F[v][u] -= flow
            path = bfs(C, F, s, t)
        return sum(F[s][i] for i in range(n))

#find path by using BFS
def bfs(C, F, s, t):
        queue = [s]
        paths = {s:[]}
        if s == t:
            return paths[s]
        while queue: 
            u = queue.pop(0)
            for v in range(len(C)):
                    if(C[u][v]-F[u][v]>0) and v not in paths:
                        paths[v] = paths[u]+[(u,v)]
                        if v == t:
                            return paths[v]
                        queue.append(v)
        return None


# Dinitz Algorithm

In [7]:
#Dinic Algorithm

#build level graph by using BFS
def Bfs(C, F, s, t):  # C is the capacity matrix
        n = len(C)
        queue = []
        queue.append(s)
        global level
        level = n * [0]  # initialization
        level[s] = 1  
        while queue:
            k = queue.pop(0)
            for i in range(n):
                    if (F[k][i] < C[k][i]) and (level[i] == 0): # not visited
                            level[i] = level[k] + 1
                            queue.append(i)
        return level[t] > 0

#search augmenting path by using DFS
def Dfs(C, F, k, cp): # k: source node, cp: support variable
        tmp = cp
        if k == len(C)-1:
            return cp
        for i in range(len(C)):
            if (level[i] == level[k] + 1) and (F[k][i] < C[k][i]):
                f = Dfs(C,F,i,min(tmp,C[k][i] - F[k][i]))
                F[k][i] = F[k][i] + f
                F[i][k] = F[i][k] - f
                tmp = tmp - f
        return cp - tmp

def dinitz(C,s,t):
        n = len(C)
        F = [n*[0] for i in range(n)] # F is the flow matrix
        flow = 0
        while(Bfs(C,F,s,t)):
               flow = flow + Dfs(C,F,s,10000000)
        return flow

# Push-relabel algorithm

In [9]:
inf = 1000000000

n = 0
capacity = []
flow = []
height = []
excess = []
seen = []
excess_vertices = []

def push(u, v):
    d = min(excess[u], capacity[u][v] - flow[u][v])
    flow[u][v] += d
    flow[v][u] -= d
    excess[u] -= d
    excess[v] += d
    if d and excess[v] == d:
        excess_vertices.append(v)

def relabel(u):
    d = inf
    for i in range(n):
        if capacity[u][i] - flow[u][i] > 0:
            d = min(d, height[i])
    if d < inf:
        height[u] = d + 1

def discharge(u):
    while excess[u] > 0:
        if seen[u] < n:
            v = seen[u]
            if capacity[u][v] - flow[u][v] > 0 and height[u] > height[v]:
                push(u, v)
            else:
                seen[u] += 1
        else:
            relabel(u)
            seen[u] = 0

def max_flow_push_relable(C, s, t):
    global n, flow, height, excess, seen, excess_vertices, capacity
    capacity = C
    n = len(capacity) # C is the capacity matrix

    height = [0] * n
    height[s] = n
    
    flow = [[0] * n for i in range(n)]
    excess = [0] * n
    excess[s] = inf
    
    for i in range(n):
        if i != s:
            push(s, i)
    seen = [0] * n

    while excess_vertices:
        u = excess_vertices.pop(0)
        if u != s and u != t:
            discharge(u)

    max_flow = 0
    for i in range(n):
        max_flow += flow[i][t]
    return max_flow

# Esecuzione degli algoritmi

In [10]:
def run_edmonds_karp(graph, source, sink):
    print(bcolors.FAIL + "Edmonds-Karp algorithm" + bcolors.ENDC)
    max_flow_value = edmonds_karp(graph, source, sink)
    print(bcolors.OKGREEN + "max_flow_value is: " + bcolors.ENDC, max_flow_value)

def run_dinic(graph, source, sink):
    print(bcolors.FAIL + "Dinic's Algorithm" + bcolors.ENDC)
    max_flow_value = dinitz(graph, source, sink)
    print(bcolors.OKGREEN + "max_flow_value is" + bcolors.ENDC, max_flow_value)

def run_push_relabel(graph, source, sink):
    print(bcolors.FAIL + "Push-Relabel algorithm" + bcolors.ENDC)
    max_flow_value = max_flow_push_relable(graph, source, sink)
    print(bcolors.OKGREEN + "max_flow_value is: " + bcolors.ENDC, max_flow_value)

In [11]:
def run_algorithms(graph):
    num_of_nodes = len(graph)
    source = num_of_nodes-2  # A
    sink = num_of_nodes-1    # F

    #dinitz
    start = datetime.datetime.now()
    run_dinic(graph, source, sink)
    end = datetime.datetime.now()
    delta = end - start
    print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    #push_relabel
    start = datetime.datetime.now()
    run_push_relabel(graph, source, sink)
    end = datetime.datetime.now()
    delta = end - start
    print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

    #ek
    start = datetime.datetime.now()
    run_edmonds_karp(graph, source, sink)
    end = datetime.datetime.now()
    delta = end - start
    print("Tempo di esecuzione: ", delta.total_seconds(), "\n")

In [ ]:
use_random_graph = False

if use_random_graph:
    for file in entries:

        graph_file = file
        print("Grafo corrente: ", graph_file, "\n")
        graph = load_graph(graph_file)

        run_algorithms(graph)

        print("\n\n")
else:
    for file in entries_prof:
        print("Grafo corrente: ", file, "\n")

        graph, source, sink = load_graph_prof(file)

        run_algorithms(graph)

        print("\n\n")


## Errore grafo 110594 nodi e 514483 archi

Grafo corrente:  BVZ-tsukuba0.max 

Dinic's Algorithm
Canceled future for execute_request message before replies were done

The Kernel crashed while executing code in the the current cell or a previous cell. Please review the code in the cell(s) to identify a possible cause of the failure. Click here for more info. View Jupyter log for further details.